In [1]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("TransitTrends- Taxi") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/04 02:54:59 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/05/04 02:54:59 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/05/04 02:54:59 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/05/04 02:54:59 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [2]:
import os
#from sodapy import Socrata
import pandas as pd
import numpy as np

In [3]:
### Loading All datasets

In [4]:
import subprocess
# downloading FHV cab data(2019-2022) from site https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_20"
output_dir = "/home/yt696_nyu_edu/TAXI/FHV/"
for i in range(19,23):
    for j in range(1, 13):
        url_1 = str(j)
        if j<10:
            url_1 = "0"+url_1
        print("wget "+ url+str(i)+"-"+url_1+".parquet")
        # Run wget command
        subprocess.call(["wget", "-P", output_dir, url+str(i)+"-"+url_1+".parquet"])

wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-01.parquet


--2023-05-04 02:55:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.23.168, 13.226.23.47, 13.226.23.72, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.23.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210707424 (201M) [application/x-www-form-urlencoded]
Saving to: ‘/home/yt696_nyu_edu/TAXI/FHV/fhv_tripdata_2019-01.parquet.5’

     0K .......... .......... .......... .......... ..........  0% 4.26M 47s
    50K .......... .......... .......... .......... ..........  0% 5.56M 42s
   100K .......... .......... .......... .......... ..........  0% 8.79M 35s
   150K .......... .......... .......... .......... ..........  0% 11.5M 31s
   200K .......... .......... .......... .......... ..........  0% 12.7M 28s
   250K .......... .......... .......... .......... ..........  0% 17.1M 25s
   300K .......... ....

wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-02.parquet


....... .......... 44%  152M 0s
  6700K .......... .......... .......... .......... .......... 45%  120M 0s
  6750K .......... .......... .......... .......... .......... 45%  152M 0s
  6800K .......... .......... .......... .......... .......... 45% 27.3M 0s
  6850K .......... .......... .......... .......... .......... 46% 64.8M 0s
  6900K .......... .......... .......... .......... .......... 46% 62.7M 0s
  6950K .......... .......... .......... .......... .......... 46%  220M 0s
  7000K .......... .......... .......... .......... .......... 47% 36.7M 0s
  7050K .......... .......... .......... .......... .......... 47% 74.3M 0s
  7100K .......... .......... .......... .......... .......... 47% 85.8M 0s
  7150K .......... .......... .......... .......... .......... 48% 69.4M 0s
  7200K .......... .......... .......... .......... .......... 48%  264M 0s
  7250K .......... .......... .......... .......... .......... 48% 46.4M 0s
  7300K .......... .......... .......... .......... ....

wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2019-03.parquet


...

.. .......... ..........  2% 13.5M 2s
   250K .......... .......... .......... .......... ..........  2% 16.1M 1s
   300K .......... .......... .......... .......... ..........  2% 13.9M 1s
   350K .......... .......... .......... .......... ..........  3% 19.8M 1s
   400K .......... .......... .......... .......... ..........  3% 24.0M 1s
   450K .......... .......... .......... .......... ..........  4% 33.5M 1s
   500K .......... .......... .......... .......... ..........  4% 26.8M 1s
   550K .......... .......... .......... .......... ..........  5% 29.4M 1s
   600K .......... .......... .......... .......... ..........  5% 60.7M 1s
   650K .......... .......... .......... .......... ..........  5% 57.2M 1s
   700K .......... .......... .......... .......... ..........  6% 32.3M 1s
   750K .......... .......... .......... .......... ..........  6% 36.9M 1s
   800K .......... .......... .......... .......... ..........  7% 70.4M 1s
   850K .......... .......... .......... .........

wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2022-11.parquet


connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298968 (11M) [binary/octet-stream]
Saving to: ‘/home/yt696_nyu_edu/TAXI/FHV/fhv_tripdata_2022-11.parquet.5’

     0K .......... .......... .......... .......... ..........  0% 4.48M 2s
    50K .......... .......... .......... .......... ..........  0% 4.75M 2s
   100K .......... .......... .......... .......... ..........  1% 9.76M 2s
   150K .......... .......... .......... .......... ..........  1% 11.3M 2s
   200K .......... .......... .......... .......... ..........  2% 13.2M 1s
   250K .......... .......... .......... .......... ..........  2% 16.1M 1s
   300K .......... .......... .......... .......... ..........  3% 18.8M 1s
   350K .......... .......... .......... .......... ..........  3% 14.0M 1s
   400K .......... .......... .......... .......... ..........  4% 23.3M 1s
   450K .......... .......... .......... .......... ..........  4% 44.6M 1s
   500K .......... .......... .......... .......... .......... 

wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2022-12.parquet


...... .......... .......... .......... .......... 31% 3.96M 0s
  4000K .......... .......... .......... .......... .......... 32% 20.9M 0s
  4050K .......... .......... .......... .......... .......... 32% 18.8M 0s
  4100K .......... .......... .......... .......... .......... 33% 20.7M 0s
  4150K .......... .......... .......... .......... .......... 33%  208M 0s
  4200K .......... .......... .......... .......... .......... 33% 18.7M 0s
  4250K .......... .......... .......... .......... .......... 34% 21.1M 0s
  4300K .......... .......... .......... .......... .......... 34% 18.9M 0s
  4350K .......... .......... .......... .......... .......... 35%  286M 0s
  4400K .......... .......... .......... .......... .......... 35% 18.4M 0s
  4450K .......... .......... .......... .......... .......... 35% 19.6M 0s
  4500K .......... .......... .......... .......... .......... 36% 33.9M 0s
  4550K .......... .......... .......... .......... .......... 36% 27.0M 0s
  4600K .......... .....

In [15]:
local_directory_path = "/home/yt696_nyu_edu/TAXI/"

# HDFS directory to copy the directory to
hdfs_directory_path = "/shared/TAXI/"
hdfs_shared_path = "/shared/"

subprocess.call(["hdfs", "dfs", "-rm", "-r", hdfs_directory_path])

rm: Permission denied by sticky bit: user=yt696_nyu_edu, path="/shared/TAXI":sk9944_nyu_edu:hadoop:drwxr-xr-x, parent="/shared":root:hadoop:drwxrwxrwt


1

23/05/04 03:03:20 WARN org.apache.spark.scheduler.cluster.YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [16]:
subprocess.call(["hdfs", "dfs", "-put", local_directory_path, hdfs_shared_path])

put: `/shared/TAXI/GREEN_CABS/green_tripdata_2021-10.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2022-09.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2021-08.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2019-12.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2020-09.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2019-02.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2022-05.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2021-07.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2019-04.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2020-01.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2019-03.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2019-10.parquet': File exists
put: `/shared/TAXI/GREEN_CABS/green_tripdata_2022-08.parquet': File exists
put: `/shared/TAXI/GREEN_

1

In [21]:
subprocess.call(["rm", "-rf", "-put", local_directory_path, hdfs_shared_path])

rm: invalid option -- 'p'
Try 'rm --help' for more information.


1

In [23]:
from pyspark.sql.functions import lit

In [24]:
fhv_df = spark.read.parquet("/shared/TAXI/FHV/", header=True, inferSchema=True)

AnalysisException: Path does not exist: hdfs://nyu-dataproc-m/shared/TAXI/FHV

In [64]:
print(fhv_df.count())

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:42873)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 33

Py4JError: An error occurred while calling o190.count

In [46]:
# Register the DataFrame as a SQL temporary view
fhv_df.createOrReplaceTempView("fhv")

In [47]:
fhv_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [63]:
# Remove duplicates
from pyspark.sql import Row
fhv_df.dropDuplicates().count()

23/05/03 22:10:44 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 15.0 in stage 42.0 (TID 568) (nyu-dataproc-sw-zz8q.c.hpc-dataproc-19b8.internal executor 2): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainLongDictionary
	at org.apache.parquet.column.Dictionary.decodeToDouble(Dictionary.java:57)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToDouble(ParquetDictionary.java:46)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getDouble(OnHeapColumnVector.java:470)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec

Py4JJavaError: An error occurred while calling o193.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 42.0 failed 10 times, most recent failure: Lost task 15.9 in stage 42.0 (TID 583) (nyu-dataproc-sw-zz8q.c.hpc-dataproc-19b8.internal executor 2): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainLongDictionary
	at org.apache.parquet.column.Dictionary.decodeToDouble(Dictionary.java:57)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToDouble(ParquetDictionary.java:46)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getDouble(OnHeapColumnVector.java:470)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.e

In [62]:
fhv_df.show()

23/05/03 22:10:22 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 15.0 in stage 41.0 (TID 542) (nyu-dataproc-sw-zz8q.c.hpc-dataproc-19b8.internal executor 2): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainLongDictionary
	at org.apache.parquet.column.Dictionary.decodeToDouble(Dictionary.java:57)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToDouble(ParquetDictionary.java:46)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getDouble(OnHeapColumnVector.java:470)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec

Py4JJavaError: An error occurred while calling o190.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 15 in stage 41.0 failed 10 times, most recent failure: Lost task 15.9 in stage 41.0 (TID 557) (nyu-dataproc-sw-zz8q.c.hpc-dataproc-19b8.internal executor 2): java.lang.UnsupportedOperationException: org.apache.parquet.column.values.dictionary.PlainValuesDictionary$PlainLongDictionary
	at org.apache.parquet.column.Dictionary.decodeToDouble(Dictionary.java:57)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetDictionary.decodeToDouble(ParquetDictionary.java:46)
	at org.apache.spark.sql.execution.vectorized.OnHeapColumnVector.getDouble(OnHeapColumnVector.java:470)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.agg_doAggregateWithKeys_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.